In [ ]:
import os

# The current working directory needs to be in explainable_TGCNN
print(os.getcwd())
os.chdir('..\\')
print(os.getcwd())

import numpy as np

import pandas as pd
from src import whole_model_demographics_gradcam, graph_plot, gc, utils, create_fake_patients
import tensorflow as tf
import math

In [ ]:
second_TGCNN_layer = True
demo = True

include_drugs = True
max_timesteps=100

stride = 1
filter_size = 4

run_name='hip_1999_to_one_year_advance_model'
years_in_advance = "5"

if include_drugs:
    max_event_codes = 518
else:
    max_event_codes = 512
hip_or_knee = 'hip'

num_filters = 16

# fake mapping dataframe for the ReadCodes and the corresponding descriptions
read_code_map_df = pd.read_csv('fake_read_code_descriptions.csv')


def remove_nans(lst):
    cleaned_list = [x for x in lst if not math.isnan(x)]
    return cleaned_list

model2 = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model2.load_weights('hip_1999_to_one_year_advance_model1_CNN_layer')

weights = model2.get_weights()

# add some random noise to each weight matrix
modified_weights = []
for w in weights:
    w_mean = np.mean(w)
    std_mean = np.std(w)
    noise = np.random.normal(w_mean, std_mean, w.shape)  # mean=0, std=0.1
    modified_w = w + noise  # Add the noise to the weights
    modified_weights.append(modified_w)

# set the modified weights back to the model
model2.set_weights(modified_weights)



model = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model.load_weights('hip_1999_to_one_year_advance_model1_CNN_layer')


num_pats = 5
cv_patients = create_fake_patients.create_fake_patient_df(num_pats, 99, max_event_codes)

In [ ]:
relu = True
filt_num = None

if relu == True:
    type = 'relu'
else:
    type = 'abs'


mse_diffs, abs_diffs = [], []

for pat_num in range(num_pats):
    if (pat_num % 50) == 0 and (pat_num !=0):
        print(f"{(((pat_num+1)/num_pats)*100):.2f}% Complete")
        mse_diffs = remove_nans(mse_diffs)
        abs_diffs = remove_nans(abs_diffs)
        print(f"MSE mean +- SD: {np.mean(mse_diffs):.3f} $\pm$ {np.std(mse_diffs):.3f}")
        print(f"Abs mean +- SD: {np.mean(abs_diffs):.3f} $\pm$ {np.std(abs_diffs):.3f}")        
        
    # Generate individual data for the model
    input_3d, input_4d, demo_tensor, outcome, outcome_bin = utils.return_pat_from_df(cv_patients, max_event_codes, hip_or_knee, pat_num, max_timesteps)
    
    dense_tensor = tf.sparse.to_dense(input_3d)
    dense_tensor= tf.transpose(dense_tensor, perm=[2, 1, 0])
    dense_tensor = np.flip(dense_tensor, axis=0)
    logits = model(input_4d, demo_tensor, training=False)
    grads = model.dy_du_branch1
    

    # ------------------------------------------------------------------------------------------------------
    heatmap_df = pd.DataFrame()
    
    # Get the entire patient's history in a DataFrame
    edges_df = graph_plot.create_edges_df_gc(dense_tensor)
    
    # Get the node positions for the graph
    pos_df = graph_plot.create_position_df_gc(edges_df)
    pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
    pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)
    
    # add the localisation map values to the heatmap too
    l_map = gc.calc_local_map(model, grads, only_pos=relu)
    
    timestep_ave_grad_df = gc.calc_timestep_weights(stride, filter_size, l_map, max_timesteps)
    
    read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_grad_df)
    # remove any rows with duplicate v number in node column
    df_unique = read_code_pos_df.drop_duplicates(subset='x', keep=False)
    
    heatmap_df[f'{type}'] = df_unique['perc_timestep_infl']
    
    heatmap1 = heatmap_df.to_numpy()

    # ------------------------------------------------------------------------------------------------------

    heatmap_df2 = pd.DataFrame()
    
    
    logits2 = model2(input_4d, demo_tensor, training=False)
    grads2 = model2.dy_du_branch1
    # add the localisation map values to the heatmap too
    l_map2 = gc.calc_local_map(model2, grads2, only_pos=relu)
    
    timestep_ave_grad_df2 = gc.calc_timestep_weights(stride, filter_size, l_map2, max_timesteps)
    
    read_code_pos_df2 = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_grad_df2)
    # remove any rows with duplicate v number in node column
    df_unique2 = read_code_pos_df2.drop_duplicates(subset='x', keep=False)
    
    heatmap_df2[f'{type}'] = df_unique2['perc_timestep_infl']
    
    heatmap2 = heatmap_df2.to_numpy()
    # ------------------------------------------------------------------------------------------------------
    mse_difference = np.mean((heatmap1 - heatmap2) ** 2)
    mse_diffs.append(mse_difference)
    abs_difference = np.mean(np.abs(heatmap1- heatmap2))
    abs_diffs.append(abs_difference)
    
mse_diffs = remove_nans(mse_diffs)
abs_diffs = remove_nans(abs_diffs)
print(f"MSE mean +- SD: {np.mean(mse_diffs):.3f} $\pm$ {np.std(mse_diffs):.3f}")
print(f"Abs mean +- SD: {np.mean(abs_diffs):.3f} $\pm$ {np.std(abs_diffs):.3f}")